In [ ]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
!unzip data.zip

In [2]:
!ls


data  data.zip	sample_data


In [3]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchsummary import summary
from torchvision import transforms, datasets
from PIL import Image
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
import numpy as np
import torch
import os

In [4]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [7]:
class HairTypeDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        
        # Assuming the classes are 'straight' and 'curly'
        self.classes = ['straight', 'curly']
        
        # Map class names to indices (0 for straight, 1 for curly)
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}
        
        # Iterate through each class folder to collect image paths and labels
        for label_name in self.classes:
            label_dir = os.path.join(data_dir, label_name)
            for img_name in os.listdir(label_dir):
                img_path = os.path.join(label_dir, img_name)
                if os.path.isfile(img_path):
                    self.image_paths.append(img_path)
                    self.labels.append(self.class_to_idx[label_name])

    def __len__(self):
        """Return the total number of images in the dataset."""
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Load image
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')  # Ensure image is in RGB format
        
        # Get the label
        label = self.labels[idx]

        # Apply any transformations, if provided
        if self.transform:
            image = self.transform(image)

        return image, label


In [5]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        # Convolutional Layer: 32 filters, 3x3 kernel, ReLU activation
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(32 * 100 * 100, 64)  
        self.fc2 = nn.Linear(64, 1)  
        
        # Sigmoid activation function for binary classification
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Convolutional Layer followed by ReLU activation and Max Pooling
        x = self.pool(self.relu(self.conv1(x)))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        
        return x
    

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model, loss function, and optimizer
model = SimpleCNN()
model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
criterion = nn.BCEWithLogitsLoss()

In [6]:
summary(model, input_size=(3, 200, 200))

# Option 2: Manual counting
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 200, 200]             896
              ReLU-2         [-1, 32, 200, 200]               0
         MaxPool2d-3         [-1, 32, 100, 100]               0
            Linear-4                   [-1, 64]      20,480,064
              ReLU-5                   [-1, 64]               0
            Linear-6                    [-1, 1]              65
           Sigmoid-7                    [-1, 1]               0
Total params: 20,481,025
Trainable params: 20,481,025
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 21.97
Params size (MB): 78.13
Estimated Total Size (MB): 100.56
----------------------------------------------------------------
Total parameters: 20481025


In [11]:
train_transforms = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

train_dataset = datasets.ImageFolder(root='data/train', transform=train_transforms)
validation_dataset = datasets.ImageFolder(root='data/test', transform=train_transforms)

# Create data loaders for batch processing
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)

In [12]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6586, Acc: 0.4888, Val Loss: 0.6559, Val Acc: 0.4876
Epoch 2/10, Loss: 0.6531, Acc: 0.4875, Val Loss: 0.6477, Val Acc: 0.4876
Epoch 3/10, Loss: 0.6492, Acc: 0.4875, Val Loss: 0.6545, Val Acc: 0.4876
Epoch 4/10, Loss: 0.6423, Acc: 0.4875, Val Loss: 0.6461, Val Acc: 0.4876
Epoch 5/10, Loss: 0.6401, Acc: 0.4938, Val Loss: 0.6439, Val Acc: 0.4876
Epoch 6/10, Loss: 0.6430, Acc: 0.4875, Val Loss: 0.6526, Val Acc: 0.4876
Epoch 7/10, Loss: 0.6434, Acc: 0.4900, Val Loss: 0.6408, Val Acc: 0.4925
Epoch 8/10, Loss: 0.6436, Acc: 0.4925, Val Loss: 0.6516, Val Acc: 0.4876
Epoch 9/10, Loss: 0.6338, Acc: 0.4900, Val Loss: 0.6478, Val Acc: 0.4925
Epoch 10/10, Loss: 0.6358, Acc: 0.4925, Val Loss: 0.6437, Val Acc: 0.4925


In [9]:
median_train_acc = np.median(history['acc'])
print(f"Median Training Accuracy: {median_train_acc:.4f}")

Median Training Accuracy: 0.4875


In [10]:
std_train_loss = np.std(history['loss'])
print(f"Standard Deviation of Training Loss: {std_train_loss:.4f}")

Standard Deviation of Training Loss: 0.0206


In [13]:
mean_test_loss = np.mean(history['val_loss'])
print(f"Mean Test Loss: {mean_test_loss:.4f}")

Mean Test Loss: 0.6485


In [ ]:
test_accuracies_last_5_epochs = history['val_acc'][5:]
average_test_acc_last_5 = np.mean(test_accuracies_last_5_epochs)
print(f"Average Test Accuracy (Epochs 6-10): {average_test_acc_last_5:.4f}")


Average Test Accuracy (Epochs 6-10): 0.4905
